# Tensorflow MNIST Beginners Tutorial

This notebook follows the [Tensorflow Beginners tutorial](https://www.tensorflow.org/tutorials/mnist/beginners/).

## Setup
First we import the libraries we'll use.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## Load MNIST Data
Then we set some constants and initialize the mnist dataset.

In [2]:
FLAGS = 'MNIST_data/'

mnist = input_data.read_data_sets(FLAGS, one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## The Softmax Regression

Then we start defining the Tensorflow variables, placeholders, loss function and train step. And we finally initialize them and start a session that will run our code.

Our evidence is defined as:

\begin{equation}
evidence_i=\sum_jW_{i,j}x_i+b_i
\end{equation}

where $W_i$ is the weights and $b_i$ is the bias for class $i$, and $j$ is an index for summing over the pixels in our input image $x$. We then convert the evidence tallies into our predicted probabilities $y$ using the "softmax" function:

\begin{equation}
y=softmax(evidence)
\end{equation}

Here softmax is serving as an "activation" or "link" function, shaping the output of our linear function into the form we want -- in this case, a probability distribution over 10 cases. You can think of it as converting tallies of evidence into probabilities of our input being in each class. It's defined as:

\begin{equation}
softmax(x)=normalize(exp(x))
\end{equation}

If you expand that equation out, you get:

\begin{equation}
softmax(x)_i=\frac{exp(x_i)}{\sum_jexp(x_j)}
\end{equation}

More compactly, we can just write:

\begin{equation}
y=softmax(Wx+b)
\end{equation}

## Implementing the Regression


In [3]:
x = tf.placeholder(tf.float32, (None, 784))

`x` isn't a specific value. It's a `placeholder`, a value that we'll input when we ask TensorFlow to run a computation. We want to be able to input any number of MNIST images, each flattened into a 784-dimensional vector. We represent this as a 2-D tensor of floating-point numbers, with a shape `[None, 784]`. (Here `None` means that a dimension can be of any length.)

We also need the weights and biases for our model. We could imagine treating these like additional inputs, but TensorFlow has an even better way to handle it: `Variable`. A `Variable` is a modifiable tensor that lives in TensorFlow's graph of interacting operations. It can be used and even modified by the computation. For machine learning applications, one generally has the model parameters be `Variable`s.

In [4]:
W = tf.Variable(tf.zeros((784, 10)))
b = tf.Variable(tf.zeros([10]))

We create these `Variable`s by giving `tf.Variable` the initial value of the `Variable`: in this case, we initialize both `W` and `b` as tensors full of zeros. Since we are going to learn `W` and `b`, it doesn't matter very much what they initially are.

Notice that `W` has a shape of `[784, 10]` because we want to multiply the 784-dimensional image vectors by it to produce 10-dimensional vectors of evidence for the difference classes. `b` has a shape of `[10]` so we can add it to the output.

Now we define our model:

In [5]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

First, we multiply `x` by `W` with the expression `tf.matmul(x, W)`. This is flipped from when we multiplied them in our equation, where we had $Wx$, as a small trick to deal with `x` being a 2D tensor with multiple inputs. We then add `b`, and finally apply `tf.nn.softmax`.

## Training
In order to train our model, we need to define what it means for the model to be good. Well, actually, in machine learning we typically define what it means for a model to be bad. We call this the cost, or the loss, and it represents how far off our model is from our desired outcome. We try to minimize that error, and the smaller the error margin, the better our model is.

One very common, very nice function to determine the loss of a model is called "cross-entropy." Cross-entropy arises from thinking about information compressing codes in information theory but it winds up being an important idea in lots of areas, from gambling to machine learning. It's defined as:

\begin{equation}
H_{y′}(y)=-\sum_iy′_ilog(y_i)
\end{equation}
Where $y$ is our predicted probability distribution, and $y′$ is the true distribution (the one-hot vector with the digit labels). In some rough sense, the cross-entropy is measuring how inefficient our predictions are for describing the truth. Going into more detail about cross-entropy is beyond the scope of this tutorial, but it's well worth [understanding](http://colah.github.io/posts/2015-09-Visual-Information/).

To implement cross-entropy we need to first add a new placeholder to input the correct answers and then we can implement the cross-entropy function, $-\sum{y′log(y)}$:

In [6]:
y_ = tf.placeholder(tf.float32, (None, 10))
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),
                                              reduction_indices=[1]))

First, `tf.log` computes the logarithm of each element of `y`. Next, we multiply each element of `y_` with the corresponding element of `tf.log(y)`. Then `tf.reduce_sum` adds the elements in the second dimension of `y`, due to the `reduction_indices=[1]` parameter. Finally, `tf.reduce_mean` computes the mean over all the examples in the batch.

Now that we know what we want our model to do, it's very easy to have TensorFlow train it to do so. Because TensorFlow knows the entire graph of your computations, it can automatically use the [backpropagation algorithm](http://colah.github.io/posts/2015-08-Backprop/) to efficiently determine how your variables affect the loss you ask it to minimize. Then it can apply your choice of optimization algorithm to modify the variables and reduce the loss.

In [7]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In this case, we ask TensorFlow to minimize `cross_entropy` using the [gradient descent algorithm](https://www.wikiwand.com/en/Gradient_descent) with a learning rate of 0.5. Gradient descent is a simple procedure, where TensorFlow simply shifts each variable a little bit in the direction that reduces the cost. But TensorFlow also provides [many other optimization algorithms](https://www.tensorflow.org/api_docs/python/train/#optimizers): using one is as simple as tweaking one line.

Now we have our model set up to train, we have to create an operation to initialize the variables we created:

In [8]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

Now we run our code in batches of 100 samples randomly for 1000 times.

> Using small batches of random data is called stochastic training -- in this case, stochastic gradient descent. Ideally, we'd like to use all our data for every step of training because that would give us a better sense of what we should be doing, but that's expensive. So, instead, we use a different subset every time. Doing this is cheap and has much of the same benefit.
> - Tensorflow

In [9]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy: {}".format(
    sess.run(accuracy,
             feed_dict={x: mnist.test.images, y_: mnist.test.labels})))

Accuracy: 0.9123000502586365
